In [1]:
import pandas as pd
import glob
from datetime import date

In [2]:
PERIODOS = [
    (1, 'INGLES I', 2412),
    (2, 'INGLES II', 2512),
    (3, 'INGLES III', 2612),
    (4, 'INGLES IV', 2712),
]
N_PARCIALES = 4
NOTA_REVISAR = 5
GRUPO_REVISAR = 'HOM'

In [3]:
# LECTURA DE NOTAS PARCIALES Y DEFINITIVAS DEL SISTEMA ACADÉMICO
SAparciales = glob.glob('./INPUT/SA_PARCIALES/*.csv')
SAdefs = glob.glob('./INPUT/SA_DEFINITIVAS/*.csv')
dfsParciales = []
dfsDef = []
for archivo in SAparciales:
    dfsParciales.append(pd.read_csv(archivo, sep=';', encoding='latin-1', skiprows=1))
for archivo in SAdefs:
    dfsDef.append(pd.read_csv(archivo, sep=';', encoding='latin-1', skiprows=1))
notasParcialesSA = pd.concat(dfsParciales, ignore_index = True)
notasDefinitivasSA = pd.concat(dfsDef, ignore_index = True)

In [4]:
# LECTURA DE ESTUDIANTES HOMOLOGADOS
estHomCiclos = glob.glob('./INPUT/E_HOMOLOGADOS/*.csv')
dfsHom = []
for archivo in estHomCiclos:
    dfsHom.append(pd.read_csv(archivo, sep=';'))
estudiantesHom = pd.concat(dfsHom, ignore_index = True)

In [5]:
# PROCESAMIENTO
# Si este bloque se ejecuta sin mostrar mensaje es porque todo está en orden.
archivoEstudiantesHom = {
    'CÓDIGO': [],
    'CÓDIGO_ASIGNATURA': [],
    'ASIGNATURA': [],
    'PERIODO': []
}
for nPeriodo, nombrePeriodo, codAsignatura in PERIODOS:
    dfParcialesPeriodo = notasParcialesSA[(notasParcialesSA['PE'] == nPeriodo) & (notasParcialesSA['ASIGNATURA'] == nombrePeriodo)]
    dfDefinitivasPeriodo = notasDefinitivasSA[(notasDefinitivasSA['PE'] == nPeriodo) & (notasDefinitivasSA['ASIGNATURA'] == nombrePeriodo)]
    dfEst = estudiantesHom[estudiantesHom['PERIODO'] == nPeriodo]
    for index, row in dfEst.iterrows():
        codEst = row['CODIGO_ESTUDIANTE']        
        dfDefEst = dfDefinitivasPeriodo[dfDefinitivasPeriodo['ESTUDIANTE'] == codEst]
        dfParcialesEst = dfParcialesPeriodo[dfParcialesPeriodo['ESTUDIANTE'] == codEst]
        if len(dfDefEst) != 1:
            print ('Error: definitiva no encontrada', codEst, 'Periodo:', nPeriodo)
            continue
        if dfDefEst['NOTA'].iloc[0] != NOTA_REVISAR:
            print ('Error: nota definitiva', codEst, 'Periodo:', nPeriodo, 'Nota:', dfDefEst['NOTA'])
        if dfDefEst['GRUPO'].iloc[0] != GRUPO_REVISAR:
            print ('Error: grupo incorrecto', codEst, 'Periodo:', nPeriodo, 'Grupo:', dfDefEst['GRUPO'])
        if len(dfParcialesEst) != N_PARCIALES:
            print ('Error: número de notas parciales no coindice', codEst, 'Periodo:', nPeriodo, '# Notas parciales:', len(dfParcialesEst))
            continue
        dfParcialesEst = dfParcialesEst.sort_values('No')
        for iP, rowP in dfParcialesEst.iterrows():
            if rowP['NOTA'] != NOTA_REVISAR:
                print ('Error: nota parcial incorrecta #', rowP['No'], codEst, 'Periodo:', nPeriodo, 'Nota: ', rowP['NOTA'])
            if rowP['GRUPO'] != GRUPO_REVISAR:
                print ('Error: grupo parcial incorrecto', codEst, 'Periodo:', nPeriodo, 'Grupo: ', rowP['GRUPO'])
        archivoEstudiantesHom['CÓDIGO'].append(codEst)
        archivoEstudiantesHom['CÓDIGO_ASIGNATURA'].append(codAsignatura)
        archivoEstudiantesHom['ASIGNATURA'].append(nombrePeriodo)
        archivoEstudiantesHom['PERIODO'].append(nPeriodo)

In [6]:
rta = pd.DataFrame(archivoEstudiantesHom)
rta.to_excel('./OUTPUT/INFORME_HOMOLOGADOS_PERIODO_' + str(date.today()) + '.xlsx', index=False)
pd.Series(rta['CÓDIGO'].unique(), name = 'CÓDIGO').to_excel('./OUTPUT/TODOS_HOMOLOGADOS_' + str(date.today()) + '.xlsx', index=False)